In [1]:
'''
데이터로드
-json로드해서 dataframe으로 변환
전처리
-null값제거, 특수문자제거(이모지?)
-cnn이 아니기때문에 형태소분리 사용하는게 좋을듯
-konlpy|twitter postagger 사용
클러스터링
-ngram방식?
시각화가 가능하다면 이해하기 쉬움
-임베딩은 word2vec/tfidf
시각화 가능하며 이해하기 쉬움
-kmeans
속도면에서 현실적인 방법
정리
-시각화, 스토리텔링
'''

'\n데이터로드\n-json로드해서 dataframe으로 변환\n전처리\n-null값제거, 특수문자제거(이모지?)\n-cnn이 아니기때문에 형태소분리 사용하는게 좋을듯\n-konlpy|twitter postagger 사용\n클러스터링\n-ngram방식?\n시각화가 가능하다면 이해하기 쉬움\n-임베딩은 word2vec/tfidf\n시각화 가능하며 이해하기 쉬움\n-kmeans\n속도면에서 현실적인 방법\n정리\n-시각화, 스토리텔링\n'

## import

In [1]:
import json
import pandas as pd
import re
from pandas.io.json import json_normalize
from konlpy.tag import Okt
from collections import Counter
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from gensim.models import Word2Vec
from collections import defaultdict
from sklearn.manifold import TSNE
from bokeh.plotting import figure, show, output_notebook

C:\Users\mediance_ssh\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## init

## data load

In [2]:
with open('../data/comment/naverstore-lingtea-comment.json','r',encoding='utf-8') as f:
    df_raw = json_normalize(json.load(f)) # json -> pandas변환
f.close()

## preprocessing

In [3]:
okt = Okt() # (구 twitter)
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
pos = ['Noun','Adjective'] # 명사, 형용사, 동사?

# 특수문자, 이모지 제거
def cleanSentences(string):
    return re.sub(re.compile("[^0-9a-zA-Z가-힣 ]"), " ", string.lower())
 
# tokenizing
def getTokenized(sentence):
    # .morphs 형태소단위분리 - norm 정규화, stem 어간추출
    # .nouns 명사추출
    # .parases 어절추출
    # .pos 품사태깅
    tokenized = okt.pos(sentence, stem=True) # 품사태깅 후 필요에따라 추출 
    tokenized = [token[0] for token in tokenized if token[1] in pos] # 해당품사만 선택
    #     tokenized = [word for word in tokenized if not word in stopwords] # 불용어 제거
    return tokenized

In [4]:
# score 분리
df_raw[['reviewScore','contents']].loc[ df_raw['reviewScore'] < 4][:3]

,reviewScore,contents
2,3,먹어도 효과는 없어요
9,2,포카리스웨터가루먹는것보다효과가없다하네요
17,3,맛 별로 구매해봤는데 가격대비 조금 비싸요 ㅠㅠ\n그래도 맛있게 잘 먹고 있습니다


In [5]:
# score별 분리 (1-3, 4-5)
df_docs = pd.DataFrame({'reviewScore':df_raw['reviewScore'] , 
                        'contents': df_raw['contents'].apply(cleanSentences).apply(getTokenized)}) # dataframe series 객체에 함수적용

docs_low = df_docs['contents'].loc[df_docs['reviewScore']<3]
docs_high = df_docs['contents'].loc[df_docs['reviewScore']>3]

print(len(docs_low), len(docs_high))

171 8743


In [6]:
docs_low[:10]

9                                 [포카리, 스웨터, 가루, 효과, 없다]
42                              [몸, 효과, 먹기, 편하다, 맛, 괜찮다]
78                             [포카리스웨트, 토레타, 그냥, 좋다, 같다]
96     [기대, 신랑, 위해, 주문, 신랑, 포, 보더, 못, 저, 음료, 그렇다, 맛, ...
124                                            [효과, 그다지]
156           [티비, 한번, 대장, 내시경, 검사, 약과, 비슷하다, 그것, 맛, 있다]
157                                   [인위, 맛, 때문, 것, 같다]
166                 [저희, 먹기, 좀, 힘, 물, 봉, 뻑뻑, 짠맛, 봉, 아쉽다]
180                                         [효과, 없다, 사지]
181            [음, 대장, 내시경, 때, 그, 약, 맛, 듯, 분, 레몬, 맛, 같다]
Name: contents, dtype: object

## analytics

#### word count

In [7]:
wordList = []
for sent in docs_low : 
    for word in sent :
        wordList.append(word)        

In [8]:
len(wordList)

1861

In [9]:
counter = Counter(wordList)
counterItems = {word:freq for word,freq in counter.items() if(freq>=1)}
counterItems = sorted(counterItems.items(), key=lambda x:x[1], reverse=True)

In [10]:
pprint(counterItems[:20])

[('맛', 93),
 ('효과', 80),
 ('없다', 49),
 ('같다', 46),
 ('좋다', 33),
 ('그냥', 25),
 ('있다', 23),
 ('더', 19),
 ('때', 19),
 ('저', 18),
 ('대장', 18),
 ('피곤하다', 17),
 ('그렇다', 16),
 ('것', 15),
 ('음료', 15),
 ('좀', 15),
 ('내시경', 15),
 ('물', 14),
 ('복숭아', 14),
 ('생각', 13)]


#### view bokeh

In [11]:
def runTsne(vectors):
    tsne = TSNE(n_components=2, perplexity=30)
    y_tsne = tsne.fit_transform(vectors)
    return y_tsne

In [12]:
def viewScatterplot(labels, title, embedding): 
    # bokeh 
    output_notebook() 
    
    # set figure
    p = figure(title=title, toolbar_location="right")
    p.grid.grid_line_color = None
    p.background_fill_color = "white"
    p.width=600
    p.height=600
    
    # scatter plot
    x=embedding[:,0]
    y=embedding[:,1]    
    p.scatter(x, y, marker="circle", size=5, line_color="navy", fill_color="orange", alpha=0.5) 
    
    # label
    for i,word in enumerate(labels) :
        x = embedding[i,0]
        y = embedding[i,1]
        p.text(x,y,text=[word], text_color="black", text_align="center", text_font_size="10pt")
    
    # show
    show(p)        

#### ngram

#### tfidf

#### word2vec

In [13]:
# 임베딩 word2vec
def word2vec(data,isDraw=True) :
    # model
    vectors = Word2Vec(data, 
                       size=200, # 임베딩차원
                       alpha=0.025, # learning rate
                       window=4, # ngram
                       min_count=10, # 최소출현빈도수
                       workers=4, # num of threads
                       sg=1, # 1이면 skipgram
#                        negative=0
                      )
        
    # tsne->bokeh
    if isDraw :
        # index
        index2word = vectors.wv.index2word
        word2index = {word:index for index, word in enumerate(index2word)}
        # bokeh
        viewScatterplot(index2word[:3000],"title",runTsne(vectors.wv.syn0))
    
    return vectors

word2vec(docs_high)

C:\Users\mediance_ssh\Anaconda3\lib\site-packages\ipykernel\__main__.py:20: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).


Loading BokehJS ...

#### k-means clustering

In [19]:
def clustering(config, data):
    
    # kmeans
    kmeans = KMeans(n_clusters=6, max_iter=10, n_init=10, verbose=0)
    vectors = word2vec(data, False)
    normVectors = normalize(vectors.wv.syn0, axis=1, norm='l2')
    clusters = kmeans.fit_predict(normVectors)
    distance = kmeans.fit_transform(normVectors)
    
    # cluster
    cluster2row = defaultdict(lambda:[])
    for rowId, label in enumerate(clusters):
        cluster2row[label].append(rowId)

    cluster2row = dict(cluster2row)
    for label, rows in cluster2row.items():
#         print('cluster #%d has %d tags' % (label, len(rows)))
        pass
        
    for label, rows in cluster2row.items():        
        print('\ncluster #%d' % label)
        for row in rows[:15]:
            print(' > %s' % vectors.wv.index2word[row])    

#### 평점별

In [20]:
clustering(None, docs_high)

C:\Users\mediance_ssh\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).



cluster #0
 > 좋다
 > 배송
 > 구매
 > 빠르다
 > 주문
 > 이다
 > 만족하다
 > 보고
 > 추천
 > 선물
 > 또
 > 아주
 > 제품
 > 일단
 > 구입

cluster #1
 > 링티
 > 링거
 > 커피
 > 힘들다
 > 하루
 > 요즘
 > 자주
 > 남편
 > 많다
 > 신랑
 > 워터
 > 체력
 > 시간
 > 일
 > 매일

cluster #2
 > 같다
 > 있다
 > 것
 > 없다
 > 생각
 > 저
 > 꾸준하다
 > 간편하다
 > 아니다
 > 가격
 > 수
 > 그렇다
 > 해
 > 전
 > 정도

cluster #3
 > 때
 > 피곤하다
 > 피로
 > 느낌
 > 거
 > 몸
 > 날
 > 정말
 > 확실하다
 > 회복
 > 숙취
 > 다음
 > 아침
 > 기분
 > 덜

cluster #4
 > 맛
 > 괜찮다
 > 더
 > 먹기
 > 물
 > 좀
 > 맛있다
 > 편하다
 > 음료
 > 복숭아
 > 레몬
 > 포카리
 > 조금
 > 요
 > 감

cluster #5
 > 효과
 > 포
 > 한번
 > 하나
 > 바로
 > 계속
 > 제
 > 번
 > 안
 > 분
 > 의사
 > 복용
 > 오늘
 > 알
 > 사서


In [21]:
clustering(None, docs_low)


cluster #0
 > 있다
 > 때
 > 저
 > 느낌
 > 생각

cluster #1
 > 그냥
 > 더
 > 그렇다
 > 복숭아
 > 개인
 > 별로
 > 피로
 > 먹기
 > 비싸다
 > 정말

cluster #2
 > 대장
 > 피곤하다
 > 가격

cluster #3
 > 효과
 > 좀
 > 물
 > 레몬
 > 똑같다
 > 제품
 > 링거

cluster #4
 > 맛
 > 같다
 > 좋다
 > 내시경
 > 포카리

cluster #5
 > 없다
 > 것
 > 음료
 > 사람
 > 기대하다


C:\Users\mediance_ssh\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).


#### bigarm

In [ ]:
'''
단어 임베딩은 word2vec이 더 진보된 방식이지만 
force-didrected 그래프로 표현하기 위해서는 
bigram을 통해 생성된 정보가 적당함
'''

In [53]:
from nltk import bigrams
from nltk.probability import ConditionalFreqDist
from nltk.util import ngrams

In [54]:
sent = ['I', 'am', 'a', 'boy']
# bigram = bigrams(sent)
# for t in bigram:
#     print(t)

tk = ConditionalFreqDist()


bigram = ngrams(sent, 2, pad_left=True, pad_right=True, left_pad_symbol='SS', right_pad_symbol='SE')
for t in bigram:
    print(t)

cfd = ConditionalFreqDist([(t[0],t[1]) for t in bigram])

('SS', 'I')
('I', 'am')
('am', 'a')
('a', 'boy')
('boy', 'SE')


In [55]:
cfd.conditions()

[]

In [56]:
cfd['SS']

FreqDist({})

## result

In [181]:
'''
평점에 따른 군집분류를 통해 핵심 주제를 뽑을 수 있음
kmeans 특징과 임베딩 알고리즘의 특징을 확인
결과의 라벨링은 사람이 해야하는데 테스트 필요

- konlpy / twitter 
- word2vec / tfidf
- kmeans
- 시각화 ngram / word2vec


word2vec 임베딩 tsne scatter가 별로인데?
ngram으로 시각화가능?
'''

'\n\n'